This notebook will highlight some of the syntax which I think is nicer in Kusto than in Pandas

First we import what we need and load the data

In [1]:
import sys
import os

import pandas as pd
%matplotlib inline
import urllib.request

In [2]:
try:
    import kusto_pandas
except:
    sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(""), '..')))
    import kusto_pandas
from kusto_pandas import Wrap

In [3]:
def dowload_dataset_if_necessary(url, filename):
    if not os.path.exists(filename):
        urllib.request.urlretrieve (url, filename)
url = "https://projects.fivethirtyeight.com/trump-approval-data/approval_polllist.csv"
filename = "approval_polllist.csv"
dowload_dataset_if_necessary(url, filename)
data = pd.read_csv(filename, parse_dates=["modeldate", "startdate", "enddate"])

In [4]:
w = Wrap(data)
w.take(2)

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Donald Trump,All polls,2021-01-20,2017-01-20,2017-01-22,Morning Consult,B/C,1992.0,rv,0.680029,...,37.0,45.686784,38.055805,NaN,NaN,http://static.politico.com/9b/13/82a3baf542ae9...,49249,77261,1/23/2017,11:47:59 20 Jan 2021
1,Donald Trump,All polls,2021-01-20,2017-01-20,2017-01-22,Gallup,B,1500.0,a,0.262323,...,45.0,45.861441,43.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,11:47:59 20 Jan 2021


# Filtering, aka the where condition

Here is how you do it in pandas

In [5]:
data[(data["grade"] == "A") | (data["grade"] == "B")].head(1)

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
1,Donald Trump,All polls,2021-01-20,2017-01-20,2017-01-22,Gallup,B,1500.0,a,0.262323,...,45.0,45.861441,43.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,11:47:59 20 Jan 2021


Problems with the Pandas syntax
1. I have to put the name of the dataframe 3 times
2. To combine two conditions I have to use the `|` or `&` operators.  In python these are bitwise operators but have been overloaded for pandas to mean elementwise or (and).  This is non intuitive and took me ages to figure out.
3. The parentheses in the above expression are mandatory otherwise it will not compute correctly.  The order of operations is confused
4. No easy way to check if "grade" is in a collection

Here is the same expression in KustoPandas

In [6]:
w.where("grade == 'A' or grade == 'B'").take(1)
# or equivalently
w.where("grade in ('A', 'B')").take(1)

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
1,Donald Trump,All polls,2021-01-20,2017-01-20,2017-01-22,Gallup,B,1500.0,a,0.262323,...,45.0,45.861441,43.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,11:47:59 20 Jan 2021


# GroupBy aggregation with rename

For something very simple, there is not much difference, but if you want to apply multiple different aggregations on various columns and give the ouput a nice name, it becomes very hard to do succinctly in pandas.  This stackoverflow question nicely illustrates the problem: https://stackoverflow.com/questions/44635626/rename-result-columns-from-pandas-aggregation-futurewarning-using-a-dict-with/50697003

Here is what we want to do in kusto

In [7]:
w.summarize("count(), NumPollsters = dcount(pollster), AverageApprovalRating = avg(approve) by grade")

,grade,count_,NumPollsters,AverageApprovalRating
0,A,83,3,41.293976
1,A+,329,8,41.051672
2,A-,261,4,43.683831
3,A/B,131,7,40.134809
4,B,5678,5,41.047693
5,B+,154,3,39.597403
6,B-,2745,4,41.047395
7,B/C,2804,21,41.357972
8,C,1010,3,45.943069
9,C+,2077,2,47.054887


Here is one way you could do it in pandas.  There are other ways as well (see stackoverflow question above) but they are not necessarily nicer.

In [8]:
grouped = data.groupby("grade")
newdf = pd.DataFrame()
newdf["count_"] = grouped.size()
newdf["NumPollsters"] = grouped["pollster"].nunique()
newdf["AverageApprovalRating"] = grouped["approve"].mean()
newdf

,count_,NumPollsters,AverageApprovalRating
grade,,,
A,83,3,41.293976
A+,329,8,41.051672
A-,261,4,43.683831
A/B,131,7,40.134809
B,5678,5,41.047693
B+,154,3,39.597403
B-,2745,4,41.047395
B/C,2804,21,41.357972
C,1010,3,45.943069


Pandas 0.25.0 (released July 2019) supports [GroupBy aggregation with relabeling](https://pandas.pydata.org/pandas-docs/stable/whatsnew/v0.25.0.html#groupby-aggregation-with-relabeling).  So the expression above could be somewhat simplified, however I still think the Kusto syntax is much much more user friendly.  Here is the example from the link above

```python
In [3]: animals.groupby("kind").agg(
   ...:     min_height=pd.NamedAgg(column='height', aggfunc='min'),
   ...:     max_height=pd.NamedAgg(column='height', aggfunc='max'),
   ...:     average_weight=pd.NamedAgg(column='weight', aggfunc=np.mean),
   ...: )
```


# GroupBy aggregation with abitrary transformations

One of the nice aspects of kusto is the ability to combine arbitrary transformations of a column, or to combine multiple columns in the summarize operator


In [9]:
w.summarize("count(), AverageAdjustedApprovalDiff = avg(adjusted_approve - approve) by bin(startdate, 1d)").take(4)

,bin_startdate,count_,AverageAdjustedApprovalDiff
0,2017-01-20,9,0.887368
1,2017-01-21,5,0.658992
2,2017-01-22,7,-0.894150
3,2017-01-23,12,-0.458746


I don't think there is any way to do this simply in pandas.  The best way I know of is this

In [10]:
# the groupby column is transformed, so we have to pre-compute that
data["BinnedStartDate"] = data["startdate"].dt.floor("1d")
# One of the aggregations is over a transformed column
data["approve_diff"] = data["adjusted_approve"] -  data["approve"]
grouped = data.groupby("BinnedStartDate")
newdf = pd.DataFrame()
newdf["count_"] = grouped.size()
newdf["AverageAdjustedApprovalDiff"] = grouped["approve_diff"].mean()
# remove the temporary columns we created
del data["BinnedStartDate"]
del data["approve_diff"]
newdf.head(4)

,count_,AverageAdjustedApprovalDiff
BinnedStartDate,,
2017-01-20,9,0.887368
2017-01-21,5,0.658992
2017-01-22,7,-0.894150
2017-01-23,12,-0.458746


# Index vs columns

In pandas after doing groupby aggregations the column that you grouped on is now an index, not a column anymore.  This has some benefits, e.g. retrieving rows with a given index, or subsequently joining on another dataframe using that index.  However it also completely changes how to interact with that "column" of the dataframe.  Even simple expressions like just accessing the data you have to do  differently


```python
newdf = data.groupby("grade")["pollster"].nunique()
newdf["grade"] # Error
```

Most other database query languages maintain the indexes behind the scenes without completely changing how you interact with that data that is the index key.

KustoPandas resets the index after groupby expressions to ensure that columns are always columns.  There will be a performance impact for some types of expressions  

In [11]:
w

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Donald Trump,All polls,2021-01-20,2017-01-20,2017-01-22,Morning Consult,B/C,1992.0,rv,0.680029,...,37.0,45.686784,38.055805,NaN,NaN,http://static.politico.com/9b/13/82a3baf542ae9...,49249,77261,1/23/2017,11:47:59 20 Jan 2021
1,Donald Trump,All polls,2021-01-20,2017-01-20,2017-01-22,Gallup,B,1500.0,a,0.262323,...,45.0,45.861441,43.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,11:47:59 20 Jan 2021
2,Donald Trump,All polls,2021-01-20,2017-01-20,2017-01-24,Ipsos,B-,1632.0,a,0.153481,...,45.2,43.451563,43.780389,NaN,T,http://polling.reuters.com/#poll/CP3_2/,49426,77599,3/1/2017,11:47:59 20 Jan 2021
3,Donald Trump,All polls,2021-01-20,2017-01-21,2017-01-23,Gallup,B,1500.0,a,0.242845,...,46.0,45.861441,44.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49262,77274,1/24/2017,11:47:59 20 Jan 2021
4,Donald Trump,All polls,2021-01-20,2017-01-22,2017-01-24,Gallup,B,1500.0,a,0.227380,...,45.0,46.861441,43.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49236,77248,1/25/2017,11:47:59 20 Jan 2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16495,Donald Trump,Voters,2021-01-20,2021-01-15,2021-01-17,Quinnipiac University,B+,1131.0,rv,1.345482,...,61.0,36.758206,59.198940,NaN,NaN,https://poll.qu.edu/national/release-detail?Re...,74214,139262,1/18/2021,11:52:51 20 Jan 2021
16496,Donald Trump,Voters,2021-01-20,2021-01-14,2021-01-18,Rasmussen Reports/Pulse Opinion Research,C+,1500.0,lv,0.569744,...,48.0,46.837601,48.988128,NaN,T,http://www.rasmussenreports.com/public_content...,74221,139287,1/19/2021,11:52:51 20 Jan 2021
16497,Donald Trump,Voters,2021-01-20,2021-01-15,2021-01-17,Morning Consult,B/C,1993.0,rv,0.093102,...,58.0,39.495997,58.343779,NaN,NaN,https://assets.morningconsult.com/wp-uploads/2...,74218,139267,1/19/2021,11:52:51 20 Jan 2021
16498,Donald Trump,Voters,2021-01-20,2021-01-16,2021-01-19,American Research Group,B,990.0,rv,1.314191,...,67.0,32.325063,63.443719,NaN,NaN,https://americanresearchgroup.com/economy/,74226,139307,1/20/2021,11:52:51 20 Jan 2021
